<a href="https://colab.research.google.com/github/kundanvallala111-coder/kundan-vallala/blob/main/MYTHOLOGY_NLP_PROJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import nltk

# Force-download everything we might need
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')   # sometimes required with WordNet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [1]:
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Is anyone there?", "Hey"],
      "responses": ["Hello, seeker of myths!", "Hi there, what mythology question do you have?"]
    },
    {
      "tag": "greek_zeus",
      "patterns": ["Who is Zeus?", "Tell me about Zeus", "Greek god Zeus"],
      "responses": ["Zeus is the king of the Greek gods, ruler of the sky and thunder."]
    }
  ]
}

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Is anyone there?', 'Hey'],
   'responses': ['Hello, seeker of myths!',
    'Hi there, what mythology question do you have?']},
  {'tag': 'greek_zeus',
   'patterns': ['Who is Zeus?', 'Tell me about Zeus', 'Greek god Zeus'],
   'responses': ['Zeus is the king of the Greek gods, ruler of the sky and thunder.']}]}

In [2]:
!pip install nltk tensorflow==2.15.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0


In [3]:
import json
import random
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
from google.colab import files
uploaded = files.upload()

Saving mythology_intense.json to mythology_intense.json


In [10]:
import os

print(os.listdir())

['.config', 'mythology_intense (1).json', 'mythology_intense.json', 'sample_data']


In [13]:
import os

for f in os.listdir():
    if f.endswith(".json"):
        print("Deleting:", f)
        os.remove(f)

Deleting: mythology_intense (1).json
Deleting: mythology_intense.json


In [17]:
import json

with open("mythology_intense.json", "r", encoding="utf-8") as f:
    print(f.read())

{ "intents": [ { "tag": "greeting", "patterns": ["Hi", "Hello", "Hey",
"Good morning"], "responses": ["Hello! How can I help you with
mythology?", "Hi there! Ask me anything about mythology."] }, { "tag":
"zeus", "patterns": ["Who is Zeus?", "Tell me about Zeus", "Zeus
story"], "responses": ["Zeus is the king of the Greek gods.", "Zeus
ruled over Mount Olympus and the sky."] }, { "tag": "thor", "patterns":
["Who is Thor?", "Tell me about Thor"], "responses": ["Thor is the Norse
god of thunder and lightning.", "Thor wields the mighty hammer
Mjolnir."] } ] }



In [19]:
import os

for f in os.listdir():
    if f.endswith(".json"):
        print("Deleting:", f)
        os.remove(f)

Deleting: mythology_intense.json


In [21]:
import json

# 1) Define your intents data in Python (you can edit / expand this)
intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning"],
            "responses": [
                "Hello! How can I help you with mythology?",
                "Hi there! Ask me anything about mythology."
            ]
        },
        {
            "tag": "zeus",
            "patterns": ["Who is Zeus?", "Tell me about Zeus", "Zeus story"],
            "responses": [
                "Zeus is the king of the Greek gods.",
                "Zeus ruled over Mount Olympus and the sky."
            ]
        },
        {
            "tag": "thor",
            "patterns": ["Who is Thor?", "Tell me about Thor"],
            "responses": [
                "Thor is the Norse god of thunder and lightning.",
                "Thor wields the mighty hammer Mjolnir."
            ]
        }
    ]
}

# 2) Write it to mythology_intense.json (clean, valid JSON)
with open("mythology_intense.json", "w", encoding="utf-8") as f:
    json.dump(intents_data, f, ensure_ascii=False, indent=2)

print("mythology_intense.json CREATED")

# 3) Read it back to confirm it loads correctly
with open("mythology_intense.json", "r", encoding="utf-8") as f:
    intents = json.load(f)

print("Loaded OK:", intents.keys())

mythology_intense.json CREATED
Loaded OK: dict_keys(['intents'])


In [22]:
with open("mythology_intense.json", "r", encoding="utf-8") as f:
    intents = json.load(f)

intents.keys()

dict_keys(['intents'])

In [24]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
import json

with open("mythology_intense.json", "r", encoding="utf-8") as f:
    intents = json.load(f)

print("Loaded:", intents.keys())

Loaded: dict_keys(['intents'])


In [29]:
import re

def tokenize(sentence):
    # lowercase + keep only words
    return re.findall(r'\b\w+\b', sentence.lower())

In [30]:
words = []
classes = []
documents = []

for intent in intents["intents"]:
    tag = intent["tag"]
    for pattern in intent["patterns"]:
        tokens = tokenize(pattern)
        words.extend(tokens)
        documents.append((tokens, tag))
    if tag not in classes:
        classes.append(tag)

words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print("WORDS:", words)
print("CLASSES:", classes)
print("DOCUMENT COUNT:", len(documents))

WORDS: ['about', 'good', 'hello', 'hey', 'hi', 'is', 'me', 'morning', 'story', 'tell', 'thor', 'who', 'zeus']
CLASSES: ['greeting', 'thor', 'zeus']
DOCUMENT COUNT: 9


In [31]:
import numpy as np
import random

training = []
output_empty = [0] * len(classes)

for tokens, tag in documents:
    bag = [1 if w in tokens else 0 for w in words]

    output_row = output_empty.copy()
    output_row[classes.index(tag)] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print(train_x.shape, train_y.shape)

(9, 13) (9, 3)


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

model = Sequential([
    Dense(128, activation='relu', input_shape=(len(train_x[0]),)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]), activation='softmax')
])

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=150, batch_size=8, verbose=1)

model.save("mythology_chatbot.h5")

print("MODEL TRAINED!")

Epoch 1/150


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.1898 - loss: 1.2577
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3056 - loss: 1.2165
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3472 - loss: 1.1242
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4630 - loss: 1.1544
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3472 - loss: 0.9915
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3472 - loss: 1.0402
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3056 - loss: 1.0949
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4213 - loss: 1.1214
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4630 - loss: 0.9185
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6528 - loss: 0.9356
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6944 - loss: 0.8360
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7685 - loss: 0.7933
E

MODEL TRAINED!


In [33]:
def bag_of_words(sentence):
    tokens = tokenize(sentence)
    bag = np.array([1 if w in tokens else 0 for w in words])
    return bag

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]

    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    return [{"intent": classes[r[0]], "prob": str(r[1])} for r in results]

def get_response(intents_list):
    if len(intents_list) == 0:
        return "I don't know that myth. Try asking differently!"

    tag = intents_list[0]['intent']
    for intent in intents["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])

In [ ]:
print("Mythology Chatbot is ready! Type 'quit' to exit.")

while True:
    msg = input("You: ")
    if msg.lower() == "quit":
        break

    ints = predict_class(msg)
    res = get_response(ints)
    print("Bot:", res)

Mythology Chatbot is ready! Type 'quit' to exit.
You: who is zeus
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Bot: Zeus is the king of the Greek gods.
You: Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Bot: Hello! How can I help you with mythology?
You: who is rama
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Bot: Zeus is the king of the Greek gods.
